# 부스팅(Boosting)
- 기본적으로는 랜덤 포래스트와 유사한다.
- 다른 점은 각 트리들이 던지는 답의 오차들을 보정할 수 있도록 오차 함수가 존재한다.
- 이를 통해 성능을 끌어 올린다.
- 너어어어어어어어어어어어어어어무 오래걸려요.ㅠㅠ(차라리 딥러닝을 쓰고 말지.ㅠㅠㅠㅠ)

### 라이브러리 설치
- pip install xgboost
- pip install lightgbm


In [1]:
# 기본

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 모델의 최적의 하이퍼 파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 분류

In [5]:
# 데이터 준비
train_df = pd.read_parquet('/content/drive/MyDrive/pca_result20.parquet')
test_df = pd.read_parquet('/content/drive/MyDrive/test데이터/test_pca.parquet')
target_df = pd.read_csv('/content/drive/MyDrive/Segment.csv')

display(train_df.columns)
display(test_df.columns)

Index(['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10',
       'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16', 'PC17', 'PC18', 'PC19',
       'PC20'],
      dtype='object')

Index(['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'p11',
       'p12', 'p13', 'p14', 'p15', 'p16', 'p17', 'p18', 'p19', 'p20'],
      dtype='object')

In [6]:
test_df.columns = train_df.columns

In [7]:
display(train_df.columns)
display(test_df.columns)

Index(['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10',
       'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16', 'PC17', 'PC18', 'PC19',
       'PC20'],
      dtype='object')

Index(['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10',
       'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16', 'PC17', 'PC18', 'PC19',
       'PC20'],
      dtype='object')

In [9]:
target_df

,기준년월,ID,Segment
0,201807,TRAIN_000000,D
1,201807,TRAIN_000001,E
2,201807,TRAIN_000002,C
3,201807,TRAIN_000003,D
4,201807,TRAIN_000004,E
...,...,...,...
2399995,201812,TRAIN_399995,E
2399996,201812,TRAIN_399996,D
2399997,201812,TRAIN_399997,C
2399998,201812,TRAIN_399998,E


In [10]:
encoder1 = LabelEncoder()
Seg = encoder1.fit_transform(target_df['Segment'])


In [11]:
Seg

array([3, 4, 2, ..., 2, 4, 4])

In [12]:
target_df

,기준년월,ID,Segment
0,201807,TRAIN_000000,D
1,201807,TRAIN_000001,E
2,201807,TRAIN_000002,C
3,201807,TRAIN_000003,D
4,201807,TRAIN_000004,E
...,...,...,...
2399995,201812,TRAIN_399995,E
2399996,201812,TRAIN_399996,D
2399997,201812,TRAIN_399997,C
2399998,201812,TRAIN_399998,E


In [13]:
# 입력과 결과로 나눈다.
X = train_df
y = Seg

In [14]:
train_X = X
train_y = y

In [15]:
test_X = test_df

### 기본 모델 사용하기

In [16]:
print(pd.Series(train_y).unique())

[3 4 2 0 1]


In [17]:
print(pd.Series(train_y).value_counts())

4    1922052
3     349242
2     127590
0        972
1        144
Name: count, dtype: int64


In [ ]:
model5 = LGBMClassifier(device='gpu', verbose=-1)

kfold = KFold(n_splits=5, shuffle=True, random_state=1)
r1 = cross_val_score(model5, train_X, train_y, scoring='f1_weighted', cv=kfold)
print(f'평균 f1 Score : {r1.mean()}')

In [ ]:
model6 = XGBClassifier(tree_method='gpu_hist', predictor='gpu_predictor',
                       n_jobs=-1, verbosity=0, use_label_encoder=False)

# 교차 검증
kfold = KFold(n_splits=10, shuffle=True, random_state=1)

r2 = cross_val_score(model6, train_X, train_y, scoring='f1_weighted', cv=kfold)
print(f'평균 f1 Score : {r2.mean()}')

| 클래스 | 샘플 수    | 기본 가중치(역수) | 완화된 가중치(로그) | 추천 가중치 (예시) |
|--------|------------|-------------------|---------------------|--------------------|
| 4      | 1,922,052  | 0.25              | 0.22                | 1                  |
| 3      | 349,242    | 1.37              | 0.95                | 1.5                |
| 2      | 127,590    | 3.76              | 1.5                 | 4                  |
| 0      | 972        | 493.9             | 6.2                 | 50                 |
| 1      | 144        | 3330.5            | 7.8                 | 50                 |


In [ ]:
### 가중치 줘보기
class_weights = {0: 50., 1: 50, 2: 4, 3: 1.5, 4: 1}

In [ ]:
model3 = LGBMClassifier(device='gpu', class_weight=class_weights, verbose=-1)

kfold = KFold(n_splits=5, shuffle=True, random_state=1)
r3 = cross_val_score(model3, train_X, train_y, scoring='f1_weighted', cv=kfold)
print(f'평균 f1 Score : {r3.mean()}')

In [ ]:
model4 = XGBClassifier(tree_method='gpu_hist', predictor='gpu_predictor',
                       n_jobs=-1, verbosity=0, use_label_encoder=False)

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

# 클래스별 가중치 예시

f1_scores = []

for fold, (train_idx, val_idx) in enumerate(kfold.split(train_X, train_y)):
    X_tr, X_val = train_X.iloc[train_idx], train_X.iloc[val_idx]
    y_tr, y_val = train_y[train_idx], train_y[val_idx]   # .iloc → []

    sample_weights = np.array([class_weights[label] for label in y_tr])

    model4.fit(X_tr, y_tr, sample_weight=sample_weights)
    preds = model4.predict(X_val)

    f1 = f1_score(y_val, preds, average='weighted')
    f1_scores.append(f1)
    print(f'Fold {fold+1} F1 Score: {f1:.4f}')


### 모델 하이퍼 파라미터 튜닝

In [ ]:
10/0

In [ ]:
# 튜닝할 하이퍼 파라미터 후보 값
# n_estimators : 트리의 개수
# learning_rate : 학습률. 오차 보정을 위해 상수 값들을 수정하는 정도
params = {
    'n_estimators' : [50, 100, 150, 200, 250, 300],
    'learning_rate' : [0.0001, 0.001, 0.01, 0.1, 1, 0, 10, 100, 1000, 10000]
}
# 사용할 모델 객체를 생성한다.
model1 = LGBMClassifier(device='gpu', verbose=-1)
# 최적의 하이퍼 파라미터를 찾는다.
kfold = KFold(n_splits=5, shuffle=True, random_state=1)
grid_clf1 = GridSearchCV(model1, param_grid=params, scoring='f1_weighted', cv=kfold)
grid_clf1.fit(train_X, train_y)
print(f'최적의 하이퍼 파라미터 : {grid_clf1.best_params_}')
print(f'최적의 모델 평균 성능 : {grid_clf1.best_score_}')

In [ ]:
# 튜닝할 하이퍼 파라미터 후보 값
# booster : 내부에서 사용할 알고리즘. gbtree - 결정트리, gblinear - 선형모델
# n_estimators : 트리의 개수
# learning_rate : 학습률. 오차 보정을 위해 상수 값들을 수정하는 정도
params = {
    'booster' : ['gbtree', 'gblinear'],
    'n_estimators' : [50, 100, 150, 200, 250, 300],
    'learning_rate' : [0.0001, 0.001, 0.01, 0.1, 1, 0, 10, 100, 1000, 10000]
}
# 사용할 모델 객체를 생성한다.
model2 = XGBClassifier(tree_method='gpu_hist', predictor='gpu_predictor',
                       n_jobs=-1, verbosity=0, use_label_encoder=False)
# 최적의 하이퍼 파라미터를 찾는다.
kfold = KFold(n_splits=10, shuffle=True, random_state=1)
grid_clf2 = GridSearchCV(model2, param_grid=params, scoring='f1_weighted', cv=kfold)
grid_clf2.fit(train_X, train_y)
print(f'최적의 하이퍼 파라미터 : {grid_clf2.best_params_}')
print(f'최적의 모델 평균 성능 : {grid_clf2.best_score_}')

### 예측을 수행한다

In [ ]:
# 예측할 데이터를 읽어온다.
df2 = pd.read_parquet('/content/drive/MyDrive/test데이터/test_pca.parquet')
df3 = df2.copy()
df4 = df3.copy()
df5 = df4.copy()
df6 = df5.copy()
df1 = df6.copy()

df3.columns = train_X.columns
df4.columns = train_X.columns
df5.columns = train_X.columns
df6.columns = train_X.columns
df3.head()

In [ ]:
df3

In [ ]:
model3.fit(train_X, train_y)

In [ ]:
model4.fit(train_X, train_y)

In [ ]:
model5.fit(train_X, train_y)

In [ ]:
model6.fit(train_X, train_y)

In [ ]:
y_pred3= model3.predict(df3)
y_pred3

In [ ]:
# 결과를 예측한다.
y_pred4= model4.predict(df4)

In [ ]:
y_pred5= model5.predict(df5)

In [ ]:
y_pred6= model6.predict(df6)

In [ ]:
# 결과 데이터를 복원한다.
result_data3 = encoder1.inverse_transform(y_pred3)
result_data4 = encoder1.inverse_transform(y_pred4)
result_data5 = encoder1.inverse_transform(y_pred5)
result_data6 = encoder1.inverse_transform(y_pred6)

In [ ]:
df6['Segment']

In [ ]:
submission_df = pd.read_parquet('/content/drive/MyDrive/test데이터/신용카드데이터_test.parquet')
submission_df3 = submission_df.copy()
submission_df4 = submission_df.copy()
submission_df5 = submission_df.copy()
submission_df6 = submission_df.copy()

In [ ]:
submission_df3 = submission_df3[['ID','기준년월']]
submission_df4 = submission_df4[['ID','기준년월']]
submission_df5 = submission_df5[['ID','기준년월']]
submission_df6 = submission_df6[['ID','기준년월']]

In [ ]:
submission_df3

In [ ]:
from pickle import FALSE
# 결과를 저장한다.

submission_df3['Segment'] = result_data3
submission_df4['Segment'] = result_data4
submission_df5['Segment'] = result_data5
submission_df6['Segment'] = result_data6


print('저장완료')

In [ ]:
submission_df6['Segment'] = result_data6

In [ ]:

submission_df6_7 = submission_df6[submission_df6['기준년월'] == 201807]
submission_df6_8 = submission_df6[submission_df6['기준년월'] == 201808]
submission_df6_9 = submission_df6[submission_df6['기준년월'] == 201809]
submission_df6_10 = submission_df6[submission_df6['기준년월'] == 201810]
submission_df6_11 = submission_df6[submission_df6['기준년월'] == 201811]
submission_df6_12 = submission_df6[submission_df6['기준년월'] == 201812]

In [ ]:
submission_df6_7.drop('기준년월', axis=1, inplace=True)
submission_df6_8.drop('기준년월', axis=1, inplace=True)
submission_df6_9.drop('기준년월', axis=1, inplace=True)
submission_df6_10.drop('기준년월', axis=1, inplace=True)
submission_df6_11.drop('기준년월', axis=1, inplace=True)
submission_df6_12.drop('기준년월', axis=1, inplace=True)

In [ ]:
submission_df6_7

In [ ]:
submission_df6_7.to_csv('/content/drive/MyDrive/test데이터/Kaggle테스트용7월.csv', index=False)
submission_df6_8.to_csv('/content/drive/MyDrive/test데이터/Kaggle테스트용8월.csv', index=False)
submission_df6_9.to_csv('/content/drive/MyDrive/test데이터/Kaggle테스트용9월.csv', index=False)
submission_df6_10.to_csv('/content/drive/MyDrive/test데이터/Kaggle테스트용10월.csv', index=False)
submission_df6_11.to_csv('/content/drive/MyDrive/test데이터/Kaggle테스트용11월.csv', index=False)
submission_df6_12.to_csv('/content/drive/MyDrive/test데이터/Kaggle테스트용12월.csv', index=False)


In [ ]:
#확인한다
submission_df6_12

In [ ]:
submission_df6

In [ ]:
# ID의 값이 몇개인지 본다.
submission_df6['ID'].value_counts()

In [ ]:
# 랜덤으로 하나씩 추출해본다
unique_random_df = submission_df6.groupby('ID', group_keys=False).sample(n=1, random_state=42)

In [ ]:
print(unique_random_df.shape)

In [ ]:
unique_random_df.value_counts()

In [ ]:
sample_sub_df = unique_random_df.copy()
sample_sub_df.drop('기준년월', axis=1, inplace=True)
sample_sub_df

In [ ]:
# 인덱스 초기화
sample_sub_df = sample_sub_df.reset_index(drop=True)

In [ ]:
# 중복된 값 있는지 한번 확인
sample_sub_df['ID'].value_counts()[sample_sub_df['ID'].value_counts() > 1]

In [ ]:
# csv로 저장한다.
sample_sub_df.to_csv('/content/drive/MyDrive/test데이터/Kaggle테스트용s.csv', index=False)

In [ ]:
sample_sub_df